In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import random
import os
import sys
from pathlib import Path
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

#used net arch from kaggle.com/nicohrubec/pytorch-multilabel-neural-network/

In [2]:
import numpy as np
from joblib import Parallel, delayed
from scipy.interpolate import interp1d
from scipy.special import erf, erfinv
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import FLOAT_DTYPES, check_array, check_is_fitted


class GaussRankScaler(BaseEstimator, TransformerMixin):
    """Transform features by scaling each feature to a normal distribution.
    Parameters
        ----------
        epsilon : float, optional, default 1e-4
            A small amount added to the lower bound or subtracted
            from the upper bound. This value prevents infinite number
            from occurring when applying the inverse error function.
        copy : boolean, optional, default True
            If False, try to avoid a copy and do inplace scaling instead.
            This is not guaranteed to always work inplace; e.g. if the data is
            not a NumPy array, a copy may still be returned.
        n_jobs : int or None, optional, default None
            Number of jobs to run in parallel.
            ``None`` means 1 and ``-1`` means using all processors.
        interp_kind : str or int, optional, default 'linear'
           Specifies the kind of interpolation as a string
            ('linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic',
            'previous', 'next', where 'zero', 'slinear', 'quadratic' and 'cubic'
            refer to a spline interpolation of zeroth, first, second or third
            order; 'previous' and 'next' simply return the previous or next value
            of the point) or as an integer specifying the order of the spline
            interpolator to use.
        interp_copy : bool, optional, default False
            If True, the interpolation function makes internal copies of x and y.
            If False, references to `x` and `y` are used.
        Attributes
        ----------
        interp_func_ : list
            The interpolation function for each feature in the training set.
        """

    def __init__(self, epsilon=1e-4, copy=True, n_jobs=None, interp_kind='linear', interp_copy=False):
        self.epsilon = epsilon
        self.copy = copy
        self.interp_kind = interp_kind
        self.interp_copy = interp_copy
        self.fill_value = 'extrapolate'
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        """Fit interpolation function to link rank with original data for future scaling
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The data used to fit interpolation function for later scaling along the features axis.
        y
            Ignored
        """
        X = check_array(X, copy=self.copy, estimator=self, dtype=FLOAT_DTYPES, force_all_finite=True)

        self.interp_func_ = Parallel(n_jobs=self.n_jobs)(delayed(self._fit)(x) for x in X.T)
        return self

    def _fit(self, x):
        x = self.drop_duplicates(x)
        rank = np.argsort(np.argsort(x))
        bound = 1.0 - self.epsilon
        factor = np.max(rank) / 2.0 * bound
        scaled_rank = np.clip(rank / factor - bound, -bound, bound)
        return interp1d(
            x, scaled_rank, kind=self.interp_kind, copy=self.interp_copy, fill_value=self.fill_value)

    def transform(self, X, copy=None):
        """Scale the data with the Gauss Rank algorithm
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The data used to scale along the features axis.
        copy : bool, optional (default: None)
            Copy the input X or not.
        """
        check_is_fitted(self, 'interp_func_')

        copy = copy if copy is not None else self.copy
        X = check_array(X, copy=copy, estimator=self, dtype=FLOAT_DTYPES, force_all_finite=True)

        X = np.array(Parallel(n_jobs=self.n_jobs)(delayed(self._transform)(i, x) for i, x in enumerate(X.T))).T
        return X

    def _transform(self, i, x):
        return erfinv(self.interp_func_[i](x))

    def inverse_transform(self, X, copy=None):
        """Scale back the data to the original representation
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data used to scale along the features axis.
        copy : bool, optional (default: None)
            Copy the input X or not.
        """
        check_is_fitted(self, 'interp_func_')

        copy = copy if copy is not None else self.copy
        X = check_array(X, copy=copy, estimator=self, dtype=FLOAT_DTYPES, force_all_finite=True)

        X = np.array(Parallel(n_jobs=self.n_jobs)(delayed(self._inverse_transform)(i, x) for i, x in enumerate(X.T))).T
        return X

    def _inverse_transform(self, i, x):
        inv_interp_func = interp1d(self.interp_func_[i].y, self.interp_func_[i].x, kind=self.interp_kind,
                                   copy=self.interp_copy, fill_value=self.fill_value)
        return inv_interp_func(erf(x))

    @staticmethod
    def drop_duplicates(x):
        is_unique = np.zeros_like(x, dtype=bool)
        is_unique[np.unique(x, return_index=True)[1]] = True
        return x[is_unique]

submit[targets] = preds
submit.loc[X_test['cp_type']=='ctl_vehicle', targets] = 0
submit.to_csv('submission.csv', index=False)

In [3]:
import ctypes
ctypes.cdll.LoadLibrary('caffe2_nvrtc.dll')

<CDLL 'caffe2_nvrtc.dll', handle 7ffae7c50000 at 0x2dada452430>

In [11]:
seed = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(seed)

In [12]:
p_min = 1e-15
p_max = 1 - p_min

def score(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    y_pred = np.clip(y_pred, p_min, p_max)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()

In [13]:
train_X = pd.read_csv('../input/lish-moa/train_features.csv', index_col='sig_id')
test_Y = pd.read_csv('../input/lish-moa/sample_submission.csv', index_col='sig_id')
train_Y = pd.read_csv('../input/lish-moa/train_targets_scored.csv', index_col='sig_id', dtype={f: test_Y.dtypes[f] for f in test_Y})
test_X = pd.read_csv('../input/lish-moa/test_features.csv', index_col='sig_id')

In [14]:
train_X.cp_time = train_X.cp_time / 24
test_X.cp_time = test_X.cp_time / 24

train_X['real_drug'] = train_X.cp_type == 'trt_cp'
test_X['real_drug'] = test_X.cp_type == 'trt_cp'

t = train_X.cp_dose.copy()
train_X.drop(columns=['cp_dose', 'cp_type'], inplace=True)
train_X['cp_dose'] = 1
train_X.loc[(t == 'D2'), 'cp_dose'] = 2

t = test_X.cp_dose.copy()
test_X.drop(columns=['cp_dose', 'cp_type'], inplace=True)
test_X['cp_dose'] = 1
test_X.loc[(t == 'D2'), 'cp_dose'] = 2

In [15]:
nfolds = 6
nstarts = 1
nepochs = 50
batch_size = 128
val_batch_size = batch_size * 4
criterion = nn.BCELoss()
kfold = MultilabelStratifiedKFold(n_splits=nfolds, random_state=517, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
class Dataset_my(Dataset):
    def __init__(self, df, targets, mode='train'):
        self.mode = mode
        #self.feats = feats_idx
        #self.data = df[:, feats_idx]
        self.data = df
        if mode=='train':
            self.targets = targets
    
    def __getitem__(self, idx):
        if self.mode == 'train':
            return torch.FloatTensor(self.data[idx]), torch.FloatTensor(self.targets[idx])
        elif self.mode == 'test':
            return torch.FloatTensor(self.data[idx]), 0
        
    def __len__(self):
        return len(self.data)

In [90]:
def train_model(cur_model, model_num, train_X_loc, train_Y_loc):
    set_seed(seed)
    train_set = Dataset_my(train_X_loc, train_Y_loc)

    dataloaders = {
        'train': DataLoader(train_set, batch_size=batch_size, shuffle=True)
    }

    model = cur_model(train_X_loc.shape[1]).to(device)
    Path(f'./saved_params/model{model_num}').mkdir(parents=True, exist_ok=True)
    checkpoint_path = f'./saved_params/model{model_num}/repeat_{1}_Fold_{1}.pt'
    optimizer = optim.Adam(model.parameters(), weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
    best_loss = {'train': np.inf}

    for epoch in range(nepochs):
        epoch_loss = {'train': 0.0}

        for phase in ['train']:
            if phase == 'train':
                model.train()

            for i, (x, y) in enumerate(dataloaders[phase]):
                x, y = x.to(device), y.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase=='train'):
                    preds = model(x)
                    loss = criterion(preds, y)

                    if phase=='train':
                        loss.backward()
                        optimizer.step()

        scheduler.step(epoch_loss['train'])
        print("Epoch {}: {}".format(epoch, loss.item()))

        if epoch_loss['train'] < best_loss['train']:
            best_loss = epoch_loss
            torch.save(model.state_dict(), checkpoint_path)

In [99]:
def predict_model(cur_model, model_num, test_X_loc):
    set_seed(seed)
    train_set = Dataset_my(test_X_loc, None, mode='test')

    dataloaders = {
        'test': DataLoader(train_set, batch_size=val_batch_size, shuffle=False)
    }

    model = cur_model(test_X_loc.shape[1]).to(device)
    checkpoint_path = f'./saved_params/model{model_num}/repeat_{1}_Fold_{1}.pt'
    model.load_state_dict(torch.load(checkpoint_path))
    model.eval()
    
    fold_preds = []

    for i, (x, y) in enumerate(dataloaders['test']):
        x, y = x.to(device), y.to(device)

        with torch.no_grad():
            batch_preds = model(x)
            fold_preds.append(batch_preds)

    fold_preds = torch.cat(fold_preds, dim=0).cpu().numpy()
    print(fold_preds.shape)
    return fold_preds

In [19]:
class Model4(nn.Module):
    def __init__(self, num_columns):
        super(Model4, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048, 1024))
        
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1024, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.sigmoid(self.dense3(x))
        
        return x

In [72]:
t = test_Y[test_X['real_drug'] == False]
for f in t:
    t[f] = 0
train_Y4 = train_Y.reset_index(drop=True).append(t)
train_X4 = train_X.reset_index(drop=True)
test_X4 = test_X

all_X4 = train_X4.append(test_X4).drop(columns=['real_drug'])

features_g = [col for col in train_X4.columns if 'g-' in col]
features_c = [col for col in train_X4.columns if 'c-' in col]

all_X4['g_sum'] = all_X4[features_g].sum(axis = 1)
all_X4['g_mean'] = all_X4[features_g].mean(axis = 1)
all_X4['g_std'] = all_X4[features_g].std(axis = 1)
all_X4['g_kurt'] = all_X4[features_g].kurtosis(axis = 1)
all_X4['g_skew'] = all_X4[features_g].skew(axis = 1)
all_X4['c_sum'] = all_X4[features_c].sum(axis = 1)
all_X4['c_mean'] = all_X4[features_c].mean(axis = 1)
all_X4['c_std'] = all_X4[features_c].std(axis = 1)
all_X4['c_kurt'] = all_X4[features_c].kurtosis(axis = 1)
all_X4['c_skew'] = all_X4[features_c].skew(axis = 1)
all_X4['gc_sum'] = all_X4[features_g + features_c].sum(axis = 1)
all_X4['gc_mean'] = all_X4[features_g + features_c].mean(axis = 1)
all_X4['gc_std'] = all_X4[features_g + features_c].std(axis = 1)
all_X4['gc_kurt'] = all_X4[features_g + features_c].kurtosis(axis = 1)
all_X4['gc_skew'] = all_X4[features_g + features_c].skew(axis = 1)



In [73]:
scaler = GaussRankScaler()
all_X4 = scaler.fit_transform(all_X4)

In [74]:
pca_transformer = PCA(687)
all_X4 = pca_transformer.fit_transform(all_X4)

In [75]:
train_X4 = all_X4[:train_X4.shape[0]]
test_X4 = all_X4[train_X4.shape[0]:]

In [76]:
train_X4 = np.vstack([train_X4, test_X4[test_X['real_drug'] == False]])

In [77]:
alpha_smoothing = 0.001
train_Y4 = (1 - alpha_smoothing) * train_Y4 + alpha_smoothing * train_Y4.mean(axis=1)[:, None]

In [85]:
train_Y4 = train_Y4.values

In [91]:
train_model(Model4, 4, train_X4, train_Y4)

Epoch 0: 0.04890693724155426
Epoch 1: 0.020866094157099724
Epoch 2: 0.021317237988114357
Epoch 3: 0.018429653719067574
Epoch     5: reducing learning rate of group 0 to 1.0000e-04.
Epoch 4: 0.016221515834331512
Epoch 5: 0.01831035129725933
Epoch 6: 0.015584965236485004
Epoch 7: 0.01988513581454754
Epoch 8: 0.01612839102745056
Epoch 9: 0.017518268898129463
Epoch 10: 0.01633915677666664
Epoch 11: 0.018052853643894196
Epoch 12: 0.018274301663041115
Epoch 13: 0.015799714252352715
Epoch 14: 0.016889508813619614
Epoch 15: 0.015291029587388039
Epoch 16: 0.015147121623158455
Epoch 17: 0.013446547091007233
Epoch 18: 0.015235004015266895
Epoch 19: 0.01599959284067154
Epoch 20: 0.014237137511372566
Epoch 21: 0.016841525211930275
Epoch 22: 0.01475224457681179
Epoch 23: 0.012747309170663357
Epoch 24: 0.013497750274837017
Epoch 25: 0.015094495378434658
Epoch 26: 0.015711262822151184
Epoch 27: 0.013944294303655624
Epoch 28: 0.013858674094080925
Epoch 29: 0.013367187231779099
Epoch 30: 0.0141111649572

In [101]:
train_P4 = predict_model(Model4, 4, train_X4)

(24172, 206)


In [102]:
train_P4[:train_X.shape[0]][train_X['real_drug'] == False] = 0

In [104]:
train_P4[train_X.shape[0]:] = 0

In [105]:
score(train_Y4, train_P4)

0.04231147858123351